In [2]:
import gzip
from Bio import SeqIO, SeqRecord
from Bio.SeqUtils import CodonAdaptationIndex
from tqdm import tqdm
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
import re
import pandas as pd
from sklearn.decomposition import PCA
import seaborn as sns
from datetime import datetime
import random

human_proteome_path = "../../datasets/raw/Homo_sapiens.GRCh38.cds.all.fa.gz"
base_path = "../../datasets/raw/spike_nuc_X.fasta.gz"
paths = []

human_cai_path = "../../datasets/human_cai"
covid_cai_path = "../../datasets/covid_cai"

pattern = re.compile("[^AatTgGcC*?]")

for i in range(0, 15):
    path = base_path.replace("X", str(i+1))
    paths.append(path)

def is_gene_valid(seq):
    if len(seq) % 3 != 0:
        return False
    if re.search(pattern, str(seq)):
        return False
    
    return True



In [3]:
human_proteome = SeqIO.parse(gzip.open(human_proteome_path, "rt"), "fasta")
human_valid_proteome = []
for seq_record in tqdm(human_proteome):
    if is_gene_valid(seq_record.seq):
        human_valid_proteome.append(seq_record)
        # cai.cai_for_gene(str(seq_record.seq))

human_bias = CodonAdaptationIndex(human_valid_proteome)
print(human_bias)

121766it [00:02, 56980.06it/s]


AAA	0.804
AAC	1.000
AAG	1.000
AAT	0.946
ACA	0.859
ACC	1.000
ACG	0.317
ACT	0.741
AGA	1.000
AGC	1.000
AGG	0.944
AGT	0.668
ATA	0.384
ATC	1.000
ATG	1.000
ATT	0.822
CAA	0.372
CAC	1.000
CAG	1.000
CAT	0.772
CCA	0.921
CCC	1.000
CCG	0.340
CCT	0.962
CGA	0.513
CGC	0.762
CGG	0.895
CGT	0.372
CTA	0.191
CTC	0.491
CTG	1.000
CTT	0.367
GAA	0.790
GAC	1.000
GAG	1.000
GAT	0.935
GCA	0.623
GCC	1.000
GCG	0.253
GCT	0.706
GGA	0.819
GGC	1.000
GGG	0.761
GGT	0.527
GTA	0.277
GTC	0.520
GTG	1.000
GTT	0.426
TAA	0.516
TAC	1.000
TAG	0.401
TAT	0.857
TCA	0.678
TCC	0.906
TCG	0.220
TCT	0.818
TGA	1.000
TGC	1.000
TGG	1.000
TGT	0.889
TTA	0.213
TTC	1.000
TTG	0.349
TTT	0.913



AttributeError: 'CodonAdaptationIndex' object has no attribute 'upper'

In [4]:
with open(human_cai_path, "w") as f:
    f.write(", ".join(map(str, list(human_bias.values()))) + "\r\n")

In [5]:
covid_valid_proteome = []
for path in paths:
    all = []
    i = 0
    with gzip.open(path, "rt") as handle:
        for seq_record in tqdm(SeqIO.parse(handle, "fasta")):
            all.append(seq_record)

        for seq in tqdm(random.sample(all, 10000)):
            if is_gene_valid(seq.seq):
                    covid_valid_proteome.append(seq)
        break

covid_bias = CodonAdaptationIndex(covid_valid_proteome)

1000000it [00:23, 43455.14it/s]
100%|██████████| 10000/10000 [00:00<00:00, 75395.49it/s]


In [6]:
with open(covid_cai_path, "w") as f:
    f.write(", ".join(map(str, list(covid_bias.values()))) + "\r\n")